In [4]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
# Define dataset paths
base_dir = "chest_xray"  # Update to your dataset location in repository
train_dir = os.path.join(base_dir, "pneumonia_train")
test_dir = os.path.join(base_dir, "pneumonia_test")

# Image parameters
IMG_SIZE = (150, 150)  # Resize images
BATCH_SIZE = 32  # Training batch size

In [6]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# No augmentation for test data, only rescaling
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Binary classification (Normal/Pneumonia)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

C:\Users\karta\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model
EPOCHS = 10  # Increase for better accuracy
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator
)

Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 132s 807ms/step - accuracy: 0.8880 - loss: 0.2692 - val_accuracy: 0.8093 - val_loss: 0.4444
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 130s 795ms/step - accuracy: 0.9115 - loss: 0.2300 - val_accuracy: 0.8606 - val_loss: 0.3631
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 117s 713ms/step - accuracy: 0.9172 - loss: 0.2050 - val_accuracy: 0.8157 - val_loss: 0.3988
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 77s 472ms/step - accuracy: 0.9258 - loss: 0.1793 - val_accuracy: 0.8317 - val_loss: 0.4278
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 77s 469ms/step - accuracy: 0.9265 - loss: 0.1868 - val_accuracy: 0.8109 - val_loss: 0.6284
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 79s 482ms/step - accuracy: 0.9302 - loss: 0.1835 - val_accuracy: 0.8077 - val_loss: 0.5177
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 80s 485ms/step - accuracy: 0.9441 - loss: 0.1670 - val_accuracy: 0.8478 - val_loss: 0.4774
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 82s 500ms/step - accuracy: 0.9407 - loss

In [18]:
# Evaluate on test data
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 182ms/step - accuracy: 0.8931 - loss: 0.4686
Test Accuracy: 90.38%


In [19]:
# Save the trained model
model_save_path = "models/pneumonia.h5"
model.save(model_save_path)
print(f"Model saved at {model_save_path}")

Model saved at models/pneumonia.h5


In [16]:
def predict_lung_disease(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    result = "Pneumonia Detected" if prediction[0][0] > 0.5 else "Normal Lungs"

    plt.imshow(img)
    plt.title(result)
    plt.axis("off")
    plt.show()

    return result